# Separação de partições com Pytorch sobre o dataset MNIST

In [ ]:
#importando  as bibliotecas
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [ ]:
# Defina a transformação para normalizar os dados
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Carregue o conjunto de dados MNIST
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Defina a proporção de divisão entre treinamento, validação e teste
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calcule o tamanho dos conjuntos de treinamento, validação e teste
train_size = int(train_ratio * len(mnist_dataset))
val_size = int(val_ratio * len(mnist_dataset))
test_size = len(mnist_dataset) - train_size - val_size

# Divida o conjunto de dados em treinamento, validação e teste
train_dataset, val_dataset, test_dataset = random_split(mnist_dataset, [train_size, val_size, test_size])

# Crie DataLoader para cada conjunto de dados
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Verifique o tamanho de cada conjunto
print(f'Tamanho do conjunto de treinamento: {len(train_loader.dataset)} amostras')
print(f'Tamanho do conjunto de validação: {len(val_loader.dataset)} amostras')
print(f'Tamanho do conjunto de teste: {len(test_loader.dataset)} amostras')


Tamanho do conjunto de treinamento: 48000 amostras
Tamanho do conjunto de validação: 6000 amostras
Tamanho do conjunto de teste: 6000 amostras


# Partições de treino e teste (ML vs DL)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
# Defina a transformação para normalizar os dados
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

#OUTRAS TRANSFORMAÇÕES POSSÍVEIS:
"""
transforms.RandomRotation(degrees=30) -> Rotaciona a imagem aleatoriamente até o máximo definido
transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2) -> Altera caracteristicas da imagem dentro dos intervalos dados
transforms.RandomHorizontalFlip(p=0.8) # Gira a imagem horizontalmente de forma randômica com base na probabilidade dada
transforms.RandomCrop((crop_height, crop_width)) -> Realiza crops aleatorios
transforms.Resize((new_height, new_width)) -> Aplica resize a todas imagens
"""


# Carregue o conjunto de dados MNIST
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Defina a proporção de divisão entre treinamento, validação e teste
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calcule o tamanho dos conjuntos de treinamento, validação e teste
train_size = int(train_ratio * len(mnist_dataset))
val_size = int(val_ratio * len(mnist_dataset))
test_size = len(mnist_dataset) - train_size - val_size

# Divida o conjunto de dados em treinamento, validação e teste
train_dataset, val_dataset, test_dataset = random_split(mnist_dataset, [train_size, val_size, test_size])

# Crie DataLoader para cada conjunto de dados
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 88466300.91it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 77651085.78it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 30219533.75it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 8120429.99it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Função para treinar e avaliar uma Máquina de Vetores de Suporte (SVM)
def train_and_evaluate_svm(train_loader, test_loader):
    train_features = []
    train_labels = []

    for data in train_loader:
        inputs, labels = data
        train_features.append(inputs.view(inputs.size(0), -1).numpy())
        train_labels.append(labels.numpy())

    train_features = np.concatenate(train_features, axis=0)
    train_labels = np.concatenate(train_labels, axis=0)

    svm_classifier = svm.SVC(kernel='linear')
    svm_classifier.fit(train_features, train_labels)

    test_features = []
    test_labels = []

    for data in test_loader:
        inputs, labels = data
        test_features.append(inputs.view(inputs.size(0), -1).numpy())
        test_labels.append(labels.numpy())

    test_features = np.concatenate(test_features, axis=0)
    test_labels = np.concatenate(test_labels, axis=0)

    predictions = svm_classifier.predict(test_features)
    accuracy = accuracy_score(test_labels, predictions)
    print(f'SVM Accuracy: {accuracy}')

In [ ]:
# Função para treinar e avaliar uma rede neural simples
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

def train_and_evaluate_nn(train_loader, val_loader, test_loader):
    model = SimpleNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    # Treinamento
    for epoch in range(5):  # Número arbitrário de épocas para este exemplo
        model.train()
        for data in train_loader:
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validação
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for data in val_loader:
                inputs, labels = data
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = val_correct / val_total
        print(f'Epoch {epoch + 1}, Validation Accuracy: {val_accuracy}')

    # Avaliação
    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    test_accuracy = test_correct / test_total
    print(f'Neural Network Test Accuracy: {test_accuracy}')

In [ ]:
# Treine e avalie SVM
print("Avaliando Máquina de Vetores de Suporte (SVM):")
train_and_evaluate_svm(train_loader, test_loader)

# Treine e avalie rede neural simples
print("\nAvaliando Rede Neural Simples:")
train_and_evaluate_nn(train_loader, val_loader, test_loader)

Avaliando Máquina de Vetores de Suporte (SVM):
SVM Accuracy: 0.9256666666666666

Avaliando Rede Neural Simples:
Epoch 1, Validation Accuracy: 0.9263333333333333
Epoch 2, Validation Accuracy: 0.9543333333333334
Epoch 3, Validation Accuracy: 0.9586666666666667
Epoch 4, Validation Accuracy: 0.9628333333333333
Epoch 5, Validation Accuracy: 0.9655
Neural Network Test Accuracy: 0.9673333333333334
